# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from common import analysis, units, morphology, treeutils
import models.Gunay2008.gunay_model as gunay_model
import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# Load custom synapse mechanisms
neuron.load_mechanisms('/home/luye/workspace/bgcellmodels/mechanisms/synapses')

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Create cell

In [ ]:
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Instantiate the cell
excluded = [] # mechanism names that will not be used
cell = gunay_model.define_cell("GUNAY2008_AXONLESS", exclude_mechs=excluded)
cell.instantiate(sim=nrnsim)
icell = cell.icell # cell instantiated in Hoc

# Fix model for GENESIS -> NEURON conversion
gunay_model.fix_comp_dimensions(cell)
# gunay_model.fix_num_compartments(cell, 100.0, nrnsim)

nseg = sum((sec.nseg for sec in icell.all))
print("Total number of compartments: {}".format(nseg))

In [ ]:
named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
for k, v in named_seclists.items():
    if len(v)==0:
        named_seclists.pop(k) # don't include empty SectionLists
    else:
        print("{} : {} sections".format(k, len(v)))

somatic = named_seclists['somatic']
dendritic = named_seclists['basal']
axonal = named_seclists['axonal']

soma = somatic[0]
axon = axonal[0]
dend = dendritic[0]

## Synapse Locations

A suitable compartment can be selected either using `treeutils.subtree_topology` or by using 
NEURON GUI -> Tools -> Distributed Mechanisms > Viewers > Shape Name

In [ ]:
# inspect topology to determine good synapse location
# print(treeutils.subtree_topology(soma, max_depth=1))

distal_sec = dendritic[31]
middle_sec = dendritic[25]
proximal_sec = dendritic[0]

print("Distal sec: " + distal_sec.name())

In [ ]:
# Show which compartments we are inserting synapses in
plt.figure()
ax = plt.subplot(111, projection='3d')

morphology.shapeplot(h, ax)
morphology.mark_locations(h, proximal_sec, 0.5, label='prox', markspec='or')
morphology.mark_locations(h, middle_sec, 0.5, label='mid', markspec='og')
morphology.mark_locations(h, distal_sec, 0.5, label='dist', markspec='ob')

plt.legend()
plt.show()

# Excitatory Synapse (STN)

## Experimental Synapse Properties

The article by <span style='color:blue;font-weight:bold'> Kita & Kitai (1991) </span> about GP physiology shows EPSPs triggered by STN stimulation in Figure 7:

- the long responses in figures A and B are not EPSPs, but IPSPs with reversed polarity because the cell was hyperpolarized below the reversal potential for GABA
- in **Fig. D, bottom trace** we see a large, long-duration EPSP likely triggered by multiple spikes and concurrent AMPA + NMDA receptor activations
    + EPSP with amplitude of 25 mV (deviation from baseline)
    + tau_rise ~= 5 ms
    + tau_decay ~= 16.5 ms
- In **Fig. A, top trace** we see a small-amplitude short EPSP before the longer-duration IPSP

The article by <span style='color:blue;font-weight:bold'> Hanson & Jaeger (2002) </span> shows EPSPs and the time course for short-term facilitation & depression in Figure 1.
- Fig. 1C, right panel shows STD effect on short time scale, and left panel on longer time scale

## Simulation Results

Multi-synapse rule of 3 synapses each with a synaptic conductance of 0.05 nS yields a PSP of the desired magnitude.

In [ ]:
# Set conductance so we get PSP of desired magnitude

spike_interval = 600.0
synapses, stims, netcons = [], [], []

for i, sec in enumerate((proximal_sec, middle_sec, distal_sec)):
    syn = h.GLUsyn(sec(0.5))
    
    stim = h.NetStim()
    stim.number = 1e9
    stim_rate = 50.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Common Synapse parameters
    nc.weight[0] = 1.0
    syn.tau_rec = 200.0
    syn.tau_facil = 800.0
    syn.U1 = P_release = 0.1 # release probability
    
    # AMPA synapse parameters
    conductance_nS = 0.025
    syn.gmax_AMPA = conductance_nS / P_release * 1e-3 # gmax is in [uS]
    syn.tau_r_AMPA = 1.0
    syn.tau_d_AMPA = 4.0
    
    # NMDA synapse parameters
    conductance_nS = 0.0
    syn.gmax_NMDA = conductance_nS / P_release * 1e-3
    syn.tau_r_AMPA = 3.7
    syn.tau_d_NMDA = 80
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Disable spiking for more accurate PSP measurement
# for sec in h.allsec():
#     sec.gmax_NaF = 0.0

# Alternative to disable spiking: hyperpolarizing current (like in article figure)
surf_factor = 0.01

istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = -0.68 * surf_factor # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_sec,
    'mid': middle_sec,
    'dist': distal_sec,
    'synGLU': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
#     'V_prox': {'var':'v', 'sec':'prox', 'loc':0.5},
#     'V_mid': {'var':'v', 'sec':'mid', 'loc':0.5},
#     'V_dist': {'var':'v', 'sec':'dist', 'loc':0.5},
    'gAMPA': {'pointp':'synGLU', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU', 'var':'g_NMDA'},
    'Rrp': {'pointp':'synGLU', 'var':'R'},
    'Use': {'pointp':'synGLU', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
vrec_soma = vec_dict.pop('V_soma')
v_soma = vrec_soma.as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

# Inhibitory Synapse

## Experimental Synapse Properties

For **GABA-B** receptors, the article by [Wu et al. (2005)](http://jpet.aspetjournals.org/content/312/3/1082) indicates that the component of the IPSC resulting from GABA-B receptors is about 15-40% of that resulting from GABA-A (GABA-A blocked -> peak current reaches 15-40% of control condition) dependent on location (proximal vs distal)

For <span style='color:red;font-weight:bold'>STR->GPe</span> inhibitory synapses, the article by <span style='color:blue;font-weight:bold'> Levine, Hull, Buchwald (1974) </span> about GP physiology shows following values in Figure 1:

- average IPSP trace shows approx. 5 mV dip from baseline
- exponential time constant for rising (dipping) phase : 66.0 ms
- exponential time constant for recovery phase : 100.0 ms

For <span style='color:red;font-weight:bold'>STR->GPe</span> inhibitory synapses, the article by <span style='color:blue;font-weight:bold'> Miguelez et al. (2012) </span> shows the following:

- Fig. 1A: IPSC_1 @ 10Hz : amplitude ~= 325 pA
- Fig. 1A: IPSC_5 @ 10Hz : amplitude ~= 825 pA

For <span style='color:red;font-weight:bold'>GPE->GPe</span> inhibitory synapses, the article by <span style='color:blue;font-weight:bold'> Miguelez et al. (2012) </span> shows the following:

- Fig. 1C: IPSC_1 @ 10Hz : amplitude ~= 950 pA
- Fig. 1C: IPSC_4 @ 10Hz : amplitude ~= 600 pA
- Fig. 5Ba: IPSC_1 in control condition : amplitude ~= 356 pA
- Fig. 5Bb: IPSC_1 in Parkinsonian condition : amplitude ~= 666 pA

## Simulation Results


In [ ]:
istim.amp = 0
istim = None

## Simulate GPe -> GPe

In [ ]:
# For GPE -> GPE: calibrate GABA-A + GABA-B simultaneously
spike_interval = 600.0
synapses, stims, netcons = [], [], []

for i, sec in enumerate((proximal_sec, middle_sec, distal_sec)):
    syn = h.GABAsyn(sec(0.5))
    
    stim = h.NetStim()
    stim.number = 10
    stim_rate = 100.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Common Synapse parameters
    nc.weight[0] = 1.0
    syn.tau_rec = 400.0
    syn.tau_facil = 1.0
    syn.U1 = P_release = 0.2 # release probability
    
    # GABA-A synapse parameters
    conductance_nS = 0.1
    syn.gmax_GABAA = conductance_nS / P_release * 1e-3 # gmax is in [uS]
    syn.tau_r_GABAA = 2.0
    syn.tau_d_GABAA = 5.0
    
    # GABA-B synapse parameters
    conductance_nS = 0.1
#     hill_factor = 0.21
    syn.gmax_GABAB = conductance_nS * 1e-3
    syn.tau_r_GABAB = 5.0 # initial species of signaling cascade
    syn.tau_d_GABAB = 10.0 # initial species of signaling cascade
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_sec,
    'mid': middle_sec,
    'dist': distal_sec,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
#     'V_prox': {'var':'v', 'sec':'prox', 'loc':0.5},
#     'V_mid': {'var':'v', 'sec':'mid', 'loc':0.5},
#     'V_dist': {'var':'v', 'sec':'dist', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
    'Itot': {'pointp':'synGABA', 'var':'i'},
    'gtot' : {'pointp':'synGABA', 'var':'g'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
vrec_soma = vec_dict.pop('V_soma')
v_soma = vrec_soma.as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

## Simulate STR -> GPe

In [ ]:
# For STR -> GPE: calibrate GABA-A + GABA-B simultaneously
spike_interval = 600.0
synapses, stims, netcons = [], [], []

for i, sec in enumerate((proximal_sec, middle_sec, distal_sec)):
    syn = h.GABAsyn(sec(0.5))
    
    stim = h.NetStim()
    stim.number = 10
    stim_rate = 10.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 750 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Common Synapse parameters
    nc.weight[0] = 1.0
    syn.tau_rec = 1.0
    syn.tau_facil = 500.0
    syn.U1 = P_release = 0.2 # release probability
    
    # GABA-A synapse parameters
    conductance_nS = 0.2 # TODO: peak conductance _after_ facilitation must equal that of GPE->GPE
    syn.gmax_GABAA = conductance_nS * 1e-3 # gmax is in [uS]
    syn.tau_r_GABAA = 2.0
    syn.tau_d_GABAA = 5.0
    
    # GABA-B synapse parameters
    conductance_nS = 0
#     hill_factor = 0.21
    syn.gmax_GABAB = conductance_nS * 1e-3
    syn.tau_r_GABAB = 5.0 # initial species of signaling cascade
    syn.tau_d_GABAB = 10.0 # initial species of signaling cascade
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_sec,
    'mid': middle_sec,
    'dist': distal_sec,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
#     'V_prox': {'var':'v', 'sec':'prox', 'loc':0.5},
#     'V_mid': {'var':'v', 'sec':'mid', 'loc':0.5},
#     'V_dist': {'var':'v', 'sec':'dist', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
    'Itot': {'pointp':'synGABA', 'var':'i'},
    'gtot' : {'pointp':'synGABA', 'var':'g'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
vrec_soma = vec_dict.pop('V_soma')
v_soma = vrec_soma.as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude

spike_interval = 0.0
synapses, stims, netcons = [], [], []

conductance_nS = 0.1
tau_rise = 66.0
tau_decay = 100.0

for i, sec in enumerate((proximal_sec,)):
    syn = h.GABAsyn(sec(0.5))
    
    stim = h.NetStim()
    stim.number = 1 # need barrage to trigger signaling cascade
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    nc.weight[0] = conductance_nS # conductance in [nS]
    # syn.gmax_AMPA = 0.01 # [uS] or set netcon weight in [nS]
    syn.gmax_GABAB = 0.0
    syn.tau_r_GABAA = tau_rise
    syn.tau_d_GABAA = tau_decay
    syn.U1 = 1.0 # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Calibrate GABAB conductance so we get PSP of desired magnitude

spike_interval = 0.0
synapses, stims, netcons = [], [], []

# NOTE: rise and decay time represent that of first quantity in signaling
#       cascade, NOT those of the conductance or IPSP
conductance_nS = 0.0375
tau_rise = 5.0 # 66.0
tau_decay = 10.0 # 100.0

for i, sec in enumerate((proximal_sec,)):
    syn = h.GABAsyn(sec(0.5))
    
    stim = h.NetStim()
    stim.number = 8 # need barrage to trigger signaling cascade
    stim_rate = 100.0 # Hz
    stim.interval = stim_rate**-1*1e3
    stim.noise = 0
    stim.start = 500 + i*spike_interval
    
    nc = h.NetCon(stim, syn)
    nc.delay = 1.0
    
    # Synapse parameters
    nc.weight[0] = conductance_nS # conductance in [nS]
#     syn.gmax_GABAB = conductance_nS * 1e-3
    syn.gmax_GABAA = 0.0
    syn.tau_r_GABAB = tau_rise
    syn.tau_d_GABAB = tau_decay
    syn.U1 = 1.0 # release probability
    
    # Save refs
    synapses.append(syn)
    stims.append(stim)
    netcons.append(nc)

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'prox': proximal_sec,
    'mid': middle_sec,
    'dist': distal_sec,
    'synGABA': synapses[0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
#     'V_prox': {'var':'v', 'sec':'prox', 'loc':0.5},
#     'V_mid': {'var':'v', 'sec':'mid', 'loc':0.5},
#     'V_dist': {'var':'v', 'sec':'dist', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 2000.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# Plot recorded traces
plt.figure()
vrec_soma = vec_dict.pop('V_soma')
v_soma = vrec_soma.as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.grid(True)

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),